In [25]:
from ib_async import *
import pandas as pd
import numpy as np
util.startLoop()

from utils.get_sector import get_sector

In [26]:
class Portfolio:
    def __init__(self, ib_client = None ,client_id = None, arctic = None):
        self.ib = self.connect_to_ib(client_id) if ib_client is None else ib_client
        self.arctic = arctic if arctic is not None else None

    def connect_to_ib(self, client_id):
        util.startLoop()
        self.ib = IB()
        clientid = client_id if client_id is not None else 0
        try:
            self.ib.connect('127.0.0.1', 7497, clientId=clientid)
            print(f'IB Connection established with clientId={clientid}')
        except ConnectionError:
            print('Failed to connect to IB API')
            self.ib = None
        return self.ib

    def get_portfolio_data(self):
        self.portfolio = pd.DataFrame(self.ib.portfolio())
        self.portfolio['symbol'] = self.portfolio['contract'].apply(lambda x: x.symbol)
        self.portfolio['asset_class'] = self.portfolio['contract'].apply(lambda x: type(x))
        return self.portfolio
    
    def get_short_put_portfolio(self):
        if not hasattr(self, 'portfolio'):
            self.get_portfolio_data()
        put_option_rows = [row for _, row in self.portfolio.iterrows() if isinstance(row['contract'], Option) and row['contract'].right == 'P' and np.sign(row['position']) == -1]
        short_put_df = pd.DataFrame(put_option_rows)
        symbols = short_put_df['symbol'].unique().tolist()

        info_dict = get_sector(symbols)
        short_put_df['sector'] = short_put_df['symbol'].apply(lambda x: info_dict[x]['sector'])
        short_put_df['market_cap'] = short_put_df['symbol'].apply(lambda x: info_dict[x]['mcap'])
        short_put_df['close'] = short_put_df['symbol'].apply(lambda x: info_dict[x]['close'])
        return short_put_df

In [27]:
self = Portfolio()

IB Connection established with clientId=0


In [4]:
sdf = self.get_short_put_portfolio()
sdf

,contract,position,marketPrice,marketValue,averageCost,unrealizedPNL,realizedPNL,account,symbol,asset_class,sector,market_cap,close
2,"Option(conId=742873258, symbol='C', lastTradeD...",-1.0,0.106909,-10.69,70.9570,60.27,0.0,U7706434,C,<class 'ib_async.contract.Option'>,Financial Services,150149.234688,78.51
10,"Option(conId=730771602, symbol='PLTR', lastTra...",-2.0,0.488264,-97.65,155.5413,213.43,0.0,U7706434,PLTR,<class 'ib_async.contract.Option'>,Technology,163493.494784,69.24
12,"Option(conId=747920714, symbol='SLB', lastTrad...",-1.0,0.152710,-15.27,49.1906,33.92,0.0,U7706434,SLB,<class 'ib_async.contract.Option'>,Energy,61541.498880,41.09
14,"Option(conId=749703800, symbol='WBS', lastTrad...",-1.0,0.613929,-61.39,69.9400,8.55,0.0,U7706434,WBS,<class 'ib_async.contract.Option'>,Financial Services,10228.034560,57.94


In [47]:
from ib_async import *
import pandas as pd

def get_latest_stock_prices(symbols, exchanges=None, currency='USD'):
    """
    Fetch latest market prices for a list of stock symbols using ib_async.
    
    Args:
        symbols (list): List of stock ticker symbols
        exchanges (dict or None): Optional dictionary mapping symbols to exchanges. 
                                  If None, defaults to 'NYSE' for all symbols
        currency (str, optional): Currency for contracts. Defaults to 'USD'
    
    Returns:
        pd.DataFrame: DataFrame with symbols and their latest prices
    """
    # Start async event loop (important for Jupyter notebooks)
    ib_async.util.startLoop()
    
    # Create connection
    ib = IB()
    ib.connect('127.0.0.1', 7497, clientId=9)

    ib.reqMarketDataType(2)
    
    # Prepare exchanges dictionary
    if exchanges is None:
        exchanges = {symbol: 'SMART' for symbol in symbols}
    
    # Store contracts and market data
    contracts = {}
    market_data = {}
    
    # Define stocks and request market data
    for ticker in symbols:
        # Use specified exchange or default to NYSE
        exchange = exchanges.get(ticker, 'NYSE')
        
        # Create contract
        contracts[ticker] = ib_async.Stock(ticker, exchange, currency)
        
        # Request market data
        market_data[ticker] = ib.reqMktData(contracts[ticker], '', False, False)
    
    # Wait for data to populate
    ib.sleep(2)
    
    # Prepare price dictionary
    prices = {}
    for ticker, data in market_data.items():
        print(data)
        # Extract last price, using close as fallback
        price = data.last if data.last is not None else data.close
        prices[ticker] = price
    
    # Clean up
    _ = [ib.cancelMktData(_c) for _c in contracts.values()]
    ib.disconnect()
    
    # Convert to DataFrame
    price_df = pd.DataFrame.from_dict(prices, orient='index', columns=['latest_price'])
    price_df.index.name = 'symbol'
    price_df.reset_index(inplace=True)
    
    return price_df


# Get unique symbols from the portfolio
symbols = df['symbol'].unique().tolist()

# Fetch latest prices
price_df = get_latest_stock_prices(symbols)

# Merge prices back to the portfolio DataFrame
df = df.merge(price_df, on='symbol', how='left')



Ticker(contract=Stock(symbol='C', exchange='SMART', currency='USD'), time=datetime.datetime(2025, 1, 18, 23, 25, 36, 376948, tzinfo=datetime.timezone.utc), marketDataType=2, minTick=0.01, bid=80.2, bidSize=6.0, ask=80.29, askSize=1.0, last=80.24, lastSize=2.0, volume=220848.0, open=78.51, high=80.45, low=78.16, close=78.51, halted=0.0, bboExchange='a60001')
Ticker(contract=Stock(symbol='PLTR', exchange='SMART', currency='USD'), time=datetime.datetime(2025, 1, 18, 23, 25, 36, 515980, tzinfo=datetime.timezone.utc), marketDataType=2, minTick=0.01, bid=72.22, bidSize=1.0, ask=72.24, askSize=10.0, last=72.24, lastSize=5.0, volume=547539.0, open=70.92, high=72.34, low=69.95, close=69.24, halted=0.0, bboExchange='9c0001')
Ticker(contract=Stock(symbol='SLB', exchange='SMART', currency='USD'), time=datetime.datetime(2025, 1, 18, 23, 25, 36, 575728, tzinfo=datetime.timezone.utc), marketDataType=2, minTick=0.01, bid=43.6, bidSize=2.0, ask=43.68, askSize=1.0, last=43.58, lastSize=1.0, volume=30731

/var/folders/4n/3q1vnyc522xc8qt12rjshzg40000gn/T/ipykernel_39723/2545061488.py:75: FutureWarning: Passing 'suffixes' which cause duplicate columns {'latest_price_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = df.merge(price_df, on='symbol', how='left')


In [50]:
price_df

,symbol,latest_price
0,C,80.24
1,PLTR,72.24
2,SLB,43.58
3,WBS,59.67


In [6]:
def get_latest_stock_prices(symbols):

    if isinstance(symbols, str):
        symbols = [symbols]
    
    contracts = [Stock(symbol=symbol) for symbol in symbols]
    prices = {}
    

Option(conId=742873258, symbol='C', lastTradeDateOrContractMonth='20250221', strike=67.5, right='P', multiplier='100', primaryExchange='AMEX', currency='USD', localSymbol='C     250221P00067500', tradingClass='C')

In [ ]:
df = self.get_portfolio_data()
df
# put_option_rows = [row for _, row in self.portfolio.iterrows() if row['contract'].right == 'P' and np.sign(row['position']) == -1]
# short_put_df = pd.DataFrame(put_option_rows)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/M6E?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=M6E&crumb=AUTX3Hs5fWm
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/6368?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=6368&crumb=AUTX3Hs5fWm


,account,contract,position,avgCost,symbol,asset_class
0,U7706434,"Option(conId=742873258, symbol='C', lastTradeD...",-1.0,70.957000,C,<class 'ib_async.contract.Option'>
1,U7706434,"Stock(conId=14016560, symbol='6368', exchange=...",200.0,7033.215000,6368,<class 'ib_async.contract.Stock'>
2,U7706434,"Option(conId=600928950, symbol='SPY', lastTrad...",1.0,1297.048300,SPY,<class 'ib_async.contract.Option'>
3,U7706434,"Option(conId=745027249, symbol='QQQ', lastTrad...",1.0,687.798300,QQQ,<class 'ib_async.contract.Option'>
4,U7706434,"Option(conId=747920714, symbol='SLB', lastTrad...",-1.0,49.190600,SLB,<class 'ib_async.contract.Option'>
5,U7706434,"Option(conId=749703800, symbol='WBS', lastTrad...",-1.0,69.940000,WBS,<class 'ib_async.contract.Option'>
6,U7706434,"Stock(conId=253190576, symbol='EWT', exchange=...",200.0,52.490000,EWT,<class 'ib_async.contract.Stock'>
7,U7706434,"Future(conId=728776011, symbol='M6E', lastTrad...",-2.0,13187.090000,M6E,<class 'ib_async.contract.Future'>
8,U7706434,"Stock(conId=444857009, symbol='PLTR', exchange...",150.0,39.018873,PLTR,<class 'ib_async.contract.Stock'>
9,U7706434,"Stock(conId=9789076, symbol='GGAL', exchange='...",130.0,71.153846,GGAL,<class 'ib_async.contract.Stock'>


In [12]:
df = sdf.copy()
df['strike'] = df['contract'].apply(lambda x: x.strike)
df['expiry'] = df['contract'].apply(lambda x: x.lastTradeDateOrContractMonth)
df['put'] = df['contract'].apply(lambda x: x.lastTradeDateOrContractMonth)+' P' +df['contract'].apply(lambda x: str(x.strike))
df['under_price'] = df['close']
df['safety_margin'] = (df['under_price'] - df['strike']) / df['under_price']
df = df[['symbol','put','under_price','position','averageCost','marketValue','unrealizedPNL','sector','market_cap','safety_margin']]
df

,symbol,put,under_price,position,averageCost,marketValue,unrealizedPNL,sector,market_cap,safety_margin
2,C,20250221 P67.5,78.51,-1.0,70.9570,-10.69,60.27,Financial Services,150149.234688,0.140237
10,PLTR,20250417 P42.0,69.24,-2.0,155.5413,-97.65,213.43,Technology,163493.494784,0.393414
12,SLB,20250131 P41.0,41.09,-1.0,49.1906,-15.27,33.92,Energy,61541.498880,0.002190
14,WBS,20250221 P55.0,57.94,-1.0,69.9400,-61.39,8.55,Financial Services,10228.034560,0.050742


In [5]:
[c for c in self.portfolio.contract if isinstance(c, Option)]

[Option(conId=742873258, symbol='C', lastTradeDateOrContractMonth='20250221', strike=67.5, right='P', multiplier='100', primaryExchange='AMEX', currency='USD', localSymbol='C     250221P00067500', tradingClass='C'),
 Option(conId=751176927, symbol='PLTR', lastTradeDateOrContractMonth='20250131', strike=91.0, right='C', multiplier='100', primaryExchange='AMEX', currency='USD', localSymbol='PLTR  250131C00091000', tradingClass='PLTR'),
 Option(conId=730771602, symbol='PLTR', lastTradeDateOrContractMonth='20250417', strike=42.0, right='P', multiplier='100', primaryExchange='AMEX', currency='USD', localSymbol='PLTR  250417P00042000', tradingClass='PLTR'),
 Option(conId=745027249, symbol='QQQ', lastTradeDateOrContractMonth='20250221', strike=490.0, right='P', multiplier='100', primaryExchange='AMEX', currency='USD', localSymbol='QQQ   250221P00490000', tradingClass='QQQ'),
 Option(conId=747920714, symbol='SLB', lastTradeDateOrContractMonth='20250131', strike=41.0, right='P', multiplier='100

In [ ]:
df = self.get_portfolio_data()
df
# put_option_rows = [row for _, row in self.portfolio.iterrows() if row['contract'].right == 'P' and np.sign(row['position']) == -1]
# short_put_df = pd.DataFrame(put_option_rows)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/M6E?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=M6E&crumb=AUTX3Hs5fWm
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/6368?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=6368&crumb=AUTX3Hs5fWm


,account,contract,position,avgCost,symbol,asset_class
0,U7706434,"Option(conId=742873258, symbol='C', lastTradeD...",-1.0,70.957000,C,<class 'ib_async.contract.Option'>
1,U7706434,"Stock(conId=14016560, symbol='6368', exchange=...",200.0,7033.215000,6368,<class 'ib_async.contract.Stock'>
2,U7706434,"Option(conId=600928950, symbol='SPY', lastTrad...",1.0,1297.048300,SPY,<class 'ib_async.contract.Option'>
3,U7706434,"Option(conId=745027249, symbol='QQQ', lastTrad...",1.0,687.798300,QQQ,<class 'ib_async.contract.Option'>
4,U7706434,"Option(conId=747920714, symbol='SLB', lastTrad...",-1.0,49.190600,SLB,<class 'ib_async.contract.Option'>
5,U7706434,"Option(conId=749703800, symbol='WBS', lastTrad...",-1.0,69.940000,WBS,<class 'ib_async.contract.Option'>
6,U7706434,"Stock(conId=253190576, symbol='EWT', exchange=...",200.0,52.490000,EWT,<class 'ib_async.contract.Stock'>
7,U7706434,"Future(conId=728776011, symbol='M6E', lastTrad...",-2.0,13187.090000,M6E,<class 'ib_async.contract.Future'>
8,U7706434,"Stock(conId=444857009, symbol='PLTR', exchange...",150.0,39.018873,PLTR,<class 'ib_async.contract.Stock'>
9,U7706434,"Stock(conId=9789076, symbol='GGAL', exchange='...",130.0,71.153846,GGAL,<class 'ib_async.contract.Stock'>


In [4]:
self.get_short_put_portfolio()

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/M6E?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=M6E&crumb=AUTX3Hs5fWm
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/6368?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=6368&crumb=AUTX3Hs5fWm


KeyError: 'symbol'

In [7]:
self.ib.portfolio()

[PortfolioItem(contract=Stock(conId=14016560, symbol='6368', right='0', primaryExchange='TSEJ', currency='JPY', localSymbol='6368.T', tradingClass='6368'), position=200.0, marketPrice=8405.0, marketValue=1681000.0, averageCost=7033.215, unrealizedPNL=274357.0, realizedPNL=0.0, account='U7706434'),
 PortfolioItem(contract=Stock(conId=3691937, symbol='AMZN', right='0', primaryExchange='ISLAND', currency='USD', localSymbol='AMZN', tradingClass='NMS'), position=20.0, marketPrice=226.30999755, marketValue=4526.2, averageCost=224.430385, unrealizedPNL=37.59, realizedPNL=0.0, account='U7706434'),
 PortfolioItem(contract=Option(conId=742873258, symbol='C', lastTradeDateOrContractMonth='20250221', strike=67.5, right='P', multiplier='100', primaryExchange='AMEX', currency='USD', localSymbol='C     250221P00067500', tradingClass='C'), position=-1.0, marketPrice=0.1069089, marketValue=-10.69, averageCost=70.957, unrealizedPNL=60.27, realizedPNL=0.0, account='U7706434'),
 PortfolioItem(contract=Sto

In [4]:
df = self.get_portfolio_data()
df
# put_option_rows = [row for _, row in self.portfolio.iterrows() if row['contract'].right == 'P' and np.sign(row['position']) == -1]
# short_put_df = pd.DataFrame(put_option_rows)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/M6E?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=M6E&crumb=AUTX3Hs5fWm
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/6368?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=6368&crumb=AUTX3Hs5fWm


,account,contract,position,avgCost,symbol,asset_class
0,U7706434,"Option(conId=742873258, symbol='C', lastTradeD...",-1.0,70.957000,C,<class 'ib_async.contract.Option'>
1,U7706434,"Stock(conId=14016560, symbol='6368', exchange=...",200.0,7033.215000,6368,<class 'ib_async.contract.Stock'>
2,U7706434,"Option(conId=600928950, symbol='SPY', lastTrad...",1.0,1297.048300,SPY,<class 'ib_async.contract.Option'>
3,U7706434,"Option(conId=745027249, symbol='QQQ', lastTrad...",1.0,687.798300,QQQ,<class 'ib_async.contract.Option'>
4,U7706434,"Option(conId=747920714, symbol='SLB', lastTrad...",-1.0,49.190600,SLB,<class 'ib_async.contract.Option'>
5,U7706434,"Option(conId=749703800, symbol='WBS', lastTrad...",-1.0,69.940000,WBS,<class 'ib_async.contract.Option'>
6,U7706434,"Stock(conId=253190576, symbol='EWT', exchange=...",200.0,52.490000,EWT,<class 'ib_async.contract.Stock'>
7,U7706434,"Future(conId=728776011, symbol='M6E', lastTrad...",-2.0,13187.090000,M6E,<class 'ib_async.contract.Future'>
8,U7706434,"Stock(conId=444857009, symbol='PLTR', exchange...",150.0,39.018873,PLTR,<class 'ib_async.contract.Stock'>
9,U7706434,"Stock(conId=9789076, symbol='GGAL', exchange='...",130.0,71.153846,GGAL,<class 'ib_async.contract.Stock'>


In [5]:
self.info_dict

{'C': {'symbol': 'C',
  'sector': 'Financial Services',
  'mcap': 150149.234688,
  'close': 78.51},
 '6368': {'symbol': '6368', 'sector': None, 'mcap': None, 'close': None},
 'SPY': {'symbol': 'SPY', 'sector': None, 'mcap': None, 'close': 591.64},
 'QQQ': {'symbol': 'QQQ', 'sector': None, 'mcap': None, 'close': 513.08},
 'SLB': {'symbol': 'SLB',
  'sector': 'Energy',
  'mcap': 61541.49888,
  'close': 41.09},
 'WBS': {'symbol': 'WBS',
  'sector': 'Financial Services',
  'mcap': 10228.03456,
  'close': 57.94},
 'EWT': {'symbol': 'EWT', 'sector': None, 'mcap': None, 'close': 51.98},
 'M6E': {'symbol': 'M6E', 'sector': None, 'mcap': None, 'close': None},
 'PLTR': {'symbol': 'PLTR',
  'sector': 'Technology',
  'mcap': 163493.494784,
  'close': 69.24},
 'GGAL': {'symbol': 'GGAL',
  'sector': 'Financial Services',
  'mcap': 11103.506432,
  'close': 69.9},
 'AMZN': {'symbol': 'AMZN',
  'sector': 'Consumer Cyclical',
  'mcap': 2375759.167488,
  'close': 220.66},
 'LOMA': {'symbol': 'LOMA',
  's

In [ ]:
df = self.get_portfolio_data()
df
# put_option_rows = [row for _, row in self.portfolio.iterrows() if row['contract'].right == 'P' and np.sign(row['position']) == -1]
# short_put_df = pd.DataFrame(put_option_rows)

AttributeError: 'numpy.ndarray' object has no attribute 'to_list'

In [ ]:
df = self.get_portfolio_data()
df
# put_option_rows = [row for _, row in self.portfolio.iterrows() if row['contract'].right == 'P' and np.sign(row['position']) == -1]
# short_put_df = pd.DataFrame(put_option_rows)

AttributeError: 'numpy.ndarray' object has no attribute 'to_list'

In [ ]:
df = self.get_portfolio_data()
df
# put_option_rows = [row for _, row in self.portfolio.iterrows() if row['contract'].right == 'P' and np.sign(row['position']) == -1]
# short_put_df = pd.DataFrame(put_option_rows)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/6368?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=6368&crumb=QqF.0bM1QIQ
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/M6E?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=M6E&crumb=AUTX3Hs5fWm


KeyError: 'mcap'

In [ ]:
df = self.get_portfolio_data()
df
# put_option_rows = [row for _, row in self.portfolio.iterrows() if row['contract'].right == 'P' and np.sign(row['position']) == -1]
# short_put_df = pd.DataFrame(put_option_rows)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/6368?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=6368&crumb=QqF.0bM1QIQ
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/M6E?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=M6E&crumb=AUTX3Hs5fWm


KeyError: 'mcap'

In [ ]:
df = self.get_portfolio_data()
df
# put_option_rows = [row for _, row in self.portfolio.iterrows() if row['contract'].right == 'P' and np.sign(row['position']) == -1]
# short_put_df = pd.DataFrame(put_option_rows)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/6368?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=6368&crumb=QqF.0bM1QIQ
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/M6E?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=M6E&crumb=AUTX3Hs5fWm


KeyError: 'mcap'

In [ ]:
df = self.get_portfolio_data()
df
# put_option_rows = [row for _, row in self.portfolio.iterrows() if row['contract'].right == 'P' and np.sign(row['position']) == -1]
# short_put_df = pd.DataFrame(put_option_rows)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/6368?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=6368&crumb=QqF.0bM1QIQ
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/M6E?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=M6E&crumb=AUTX3Hs5fWm


KeyError: 'mcap'

In [ ]:
df = self.get_portfolio_data()
df
# put_option_rows = [row for _, row in self.portfolio.iterrows() if row['contract'].right == 'P' and np.sign(row['position']) == -1]
# short_put_df = pd.DataFrame(put_option_rows)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/6368?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=6368&crumb=QqF.0bM1QIQ
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/M6E?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=M6E&crumb=AUTX3Hs5fWm


KeyError: 'mcap'

In [ ]:
df = self.get_portfolio_data()
df
# put_option_rows = [row for _, row in self.portfolio.iterrows() if row['contract'].right == 'P' and np.sign(row['position']) == -1]
# short_put_df = pd.DataFrame(put_option_rows)

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/6368?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=6368&crumb=QqF.0bM1QIQ
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/M6E?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=M6E&crumb=AUTX3Hs5fWm


KeyError: 'mcap'

In [20]:
short_put_df

,account,contract,position,avgCost,symbol,asset_class
0,U7706434,"Option(conId=742873258, symbol='C', lastTradeD...",-1.0,70.9570,C,<class 'ib_async.contract.Option'>
4,U7706434,"Option(conId=747920714, symbol='SLB', lastTrad...",-1.0,49.1906,SLB,<class 'ib_async.contract.Option'>
5,U7706434,"Option(conId=749703800, symbol='WBS', lastTrad...",-1.0,69.9400,WBS,<class 'ib_async.contract.Option'>
13,U7706434,"Option(conId=730771602, symbol='PLTR', lastTra...",-2.0,155.5413,PLTR,<class 'ib_async.contract.Option'>
